In [39]:
import pandas as pd
import numpy as np
# import abatar

In [40]:
# Conversion Metric
n = 10000
def conversion_metric():
    effect_size = 0.1
    n1, p1 = n, 0.2
    n2, p2 = n, p1*(1+effect_size)

    control = np.array([0] * int(n1 - n1 * p1) + [1] * int(n1 * p1))
    target = np.array([0] * int(n2 - n2 * p2) + [1] * int(n2 * p2))

    return control, target

# Gaussian Metric
def gaussian_metric():
    effect_size = 0.05 #%
    muC = 100
    muT = (1 + effect_size) * muC
    var = 0.8 * muC
    
    control = np.random.normal(loc=muC, scale=np.sqrt(var), size=n)
    target = np.random.normal(loc=muT, scale=np.sqrt(var), size=n)

    return control, target

# Sample SB data
def sb_metrics():
    df = pd.read_csv("sample_sb_data.csv").sort_values('variant')
    return df['turnover'].values, df['etv'].values, df['id'].values
sb_metrics()[0].shape


(20000,)

In [41]:
df = pd.DataFrame({'variant': ['Control']*n+['Target']*n
, 'conversion_metric': np.concatenate(conversion_metric(), axis=0)
, 'gaussian_metric': np.concatenate(gaussian_metric(), axis=0)
, 'sb_turnover': sb_metrics()[0]
, 'sb_etv': sb_metrics()[1]
, 'id': sb_metrics()[2]
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   variant            20000 non-null  object 
 1   conversion_metric  20000 non-null  int64  
 2   gaussian_metric    20000 non-null  float64
 3   sb_turnover        20000 non-null  float64
 4   sb_etv             20000 non-null  float64
 5   id                 20000 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 937.6+ KB


In [42]:
df.groupby("variant").describe()

conversion_metric                                           \
                    count  mean       std  min  25%  50%  75%  max   
variant                                                              
Control           10000.0  0.20  0.400020  0.0  0.0  0.0  0.0  1.0   
Target            10000.0  0.22  0.414267  0.0  0.0  0.0  0.0  1.0   

        gaussian_metric              ...   sb_etv                    id  \
                  count        mean  ...      75%          max    count   
variant                              ...                                  
Control         10000.0   99.927786  ...  12.3054   969.300701  10000.0   
Target          10000.0  105.035402  ...  12.8351  2650.172802  10000.0   

                                                                      \
                 mean           std       min         25%        50%   
variant                                                                
Control  4.156987e+06  2.288468e+06  103599.0  2202417.00  4108593.0   
Target   4.233916e+06  2.325309e+06  106957.0  2193337.75  4364329.0   

                                
                75%        max  
variant                         
Control  5403228.25  8794836.0  
Target   5448162.75  8797955.0  

[2 rows x 40 columns]

In [43]:
df.head()

,variant,conversion_metric,gaussian_metric,sb_turnover,sb_etv,id
0,Control,0,106.897069,0.0,0.000000,6960246
1,Control,0,96.568337,0.0,0.000000,5380217
2,Control,0,91.828457,0.0,0.000000,7120101
3,Control,0,98.112333,0.0,0.000000,1880287
4,Control,0,90.075718,394.0,-7.278198,1018032


In [44]:
df.to_csv("sample_test_data.csv", index=False)

In [45]:
from abatar import SimulatePower

In [46]:
res = SimulatePower(
        df['gaussian_metric'],
        sample_size=5000,
        effect_size=0.05,
        target_frac=0.5,
        alpha=0.05,
        test_stat='mean',
        n_iters=1000,
        n_permutations=500,
    )

res.get_parametric_est()
res.simulate()
print(res)

---- Tentative sample size (power = 0.8) --
t-test sample                 104
---- Power Estimation --------------
Sample size                   5000
Effect size                   0.05
Significance level (alpha)    0.05
Simulated Power               1.00
------------------------------------

